# Random Forest

In [181]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

## Load Files

In [182]:
df = pd.read_csv('/Users/hannah-ann/PycharmProjects/cosmetic-ingredient-classifier-app/data/cleaned/cleaned_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ingredients  763 non-null    object
 1   Combination  763 non-null    int64 
 2   Dry          763 non-null    int64 
 3   Normal       763 non-null    int64 
 4   Oily         763 non-null    int64 
 5   Sensitive    763 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 35.9+ KB


## Prepare Data - Feature Extraction Vectorise

In [183]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Ingredients'])
y = df[['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']]

## Split Data

In [184]:
# Split the data (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Train Classifier Modell

In [185]:
# Create the Random Forest Classifier model
model = RandomForestClassifier(n_estimators=150, random_state=36)

# Train the model on the training data
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=150, random_state=36)

## Evaluation Metric

In [186]:
# predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.6928104575163399


AttributeError: 'DataFrame' object has no attribute 'argmax'